In [ ]:
import pyemu
from pathlib import Path
import shutil 
import sys

In [ ]:
#### Base directory for runs
parent_run_path = Path("../pycap_runs")
base_run_path = parent_run_path / "pycap_base"
pest_path = parent_run_path / "pycap_pest"
template_path = pest_path / "pycap_template"

shutil.copy2('../scripts/run_pycap_ppw.py', template_path / 'run_pycap_ppw.py')

sys.path.insert(0, str(template_path))
print(sys.path)
from run_pycap_ppw import ppw_worker_pycap


In [ ]:
ls ../pycap_runs/pycap_pest/pycap_template

In [ ]:

pyemu.os_utils.start_workers(
        worker_dir=str(template_dir), 
        exe_rel_path=pestpp_ex,
        pst_rel_path='prior_mc.pst', 
        num_workers=20,
        worker_root='./', 
        master_dir='MASTER')
